In [1]:
!pip install qiskit[visualization]==1.1.0
# Use the following if you are on MacOS/zsh
#!pip install 'qiskit[visualization]'==1.1.0
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install prototype-zne
#!pip install git+https://github.com/qiskit-community/Quantum-Challenge-Grader.git
%pip install git+https://github.com/ryanhill1/Quantum-Challenge-Grader.git

  Cloning https://github.com/ryanhill1/Quantum-Challenge-Grader.git to /tmp/pip-req-build-r6aks59g
  Running command git clone --filter=blob:none --quiet https://github.com/ryanhill1/Quantum-Challenge-Grader.git /tmp/pip-req-build-r6aks59g
  Resolved https://github.com/ryanhill1/Quantum-Challenge-Grader.git to commit a9baa682d3a07450befd4db4453a7590c7d446c0
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [1]:
### Save API Token, if needed
%set_env QXToken= eafe6b19915883c206e07330426ebeec6281dd5f8cb1462262da6e8c2c76c89c315fa361276dc4e8c2679cd1bc2031cb73c747fea64ab964e0e785ca2f912232
# Make sure there is no space between the equal sign
# and the beginning of your token
# Make sure you do NOT ADD QUOTATION MARKS!!!!

env: QXToken=eafe6b19915883c206e07330426ebeec6281dd5f8cb1462262da6e8c2c76c89c315fa361276dc4e8c2679cd1bc2031cb73c747fea64ab964e0e785ca2f912232


In [2]:
# import of required libraries and modules
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# Transpiler Passes
## Layout passes
from qiskit.transpiler.passes.layout.csp_layout import CSPLayout
from qiskit.transpiler.passes.layout.dense_layout import DenseLayout
from qiskit.transpiler.passes.layout.sabre_layout import SabreLayout
from qiskit.transpiler.passes.layout.vf2_layout import VF2Layout
from qiskit.transpiler.passes.layout.trivial_layout import TrivialLayout

## Routing passes
from qiskit.transpiler.passes.routing.basic_swap import BasicSwap
from qiskit.transpiler.passes.routing.lookahead_swap import LookaheadSwap
from qiskit.transpiler.passes.routing.sabre_swap import SabreSwap
from qiskit.transpiler.passes.routing.stochastic_swap import StochasticSwap
from qiskit.transpiler.passes.routing.star_prerouting import StarPreRouting

## Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis.basis_translator import BasisTranslator
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
### The next pass could also be considered an optimization pass.
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis

## Optimization passes
from qiskit.transpiler.passes.optimization.collect_1q_runs import Collect1qRuns
from qiskit.transpiler.passes.optimization.collect_2q_blocks import Collect2qBlocks
from qiskit.transpiler.passes.optimization.consolidate_blocks import ConsolidateBlocks
from qiskit.transpiler.passes.optimization.commutative_cancellation import CommutativeCancellation

In [3]:
# get an abstract quantum circuit from Qiskit's library of circuits 
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [4]:
from qiskit import QuantumCircuit

def get_qc_characteristics(qc):
    # Determine the quantum circuit depth of `qc` and assign it to `depth`
    depth = qc.depth()
    
    # Determine the number of qubits in `qc` and assign it to `num_qubits`    
    num_qubits = qc.num_qubits
    
    # Determine the operations in `qc` and assign it to `ops`
    ops = qc.count_ops()
    
    # Determine the number of n-qubit operations (with n larger than 1) in `qc` and assign it to `num_multi_qubit_ops`
    num_multi_qubit_ops = sum(1 for op in qc.data if len(op[1]) > 1)
    
    # Return the results as a dictionary
    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}

In [5]:
# Grade your work!
grade_lab1_ex1(get_qc_characteristics)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [6]:
# print quantum circuit characteristics
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])
    
print_qc_characteristics(qc)

Quantum circuit characteristics
  Depth: 1
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1


In [7]:
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)

┌───┐
q_0: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────■──────────────────────────────────────■──────────────────────────────────■───────────────────────────■───────────────────────■──────────────────■─────────────■────────■───────┤ H ├
                                                                                                                                                                                                                                                                     │                                            │                                      │                                  │                           │                       │                  │       ┌───┐ │        │P(π/2) └───┘
q_1: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────■──────────┼──────────────────────────────────■─────────┼────────────────────────────■─────────┼────────────────────────■─────────┼──────────────────■────────┼──────────────■────────┼─────────■────────┼───────┤ H ├─┼────────■────────────
                                                                                                                                                                                                                   │                                      │          │                                  │         │                            │         │                        │         │                  │        │        ┌───┐ │        │         │P(π/2)  │       └───┘ │P(π/4)               
q_2: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────■─────────┼────────────────────────────■─────────┼──────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼───────────────■────────┼─────────┼─────────■────────┼────────┼────────┤ H ├─┼────────┼─────────■────────┼─────────────■─────────────────────
                                                                                                                                                                      │                                  │         │                            │         │          │                        │         │         │                   │        │         │         ┌───┐ │        │         │         │P(π/2)  │        │        └───┘ │P(π/4)  │                  │P(π/8)                             
q_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────■─────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼──────────┼───────────────■────────┼─────────┼─────────┼──────────■────────┼────────┼─────────┼─────────┤ H ├─┼────────┼─────────┼─────────■────────┼────────┼──────────────■────────┼──────────────────■───────────────────────────────────
                                                                                                                                │                           │         │                        │         │         │                   │        │         │          │         ┌───┐ │        │         │         │          │P(π/2)  │        │         │         └───┘ │P(π/4)  │         │                  │P(π/8)  │                       │P(π/16)                                               
q_4: ─────────────────────────────────────────────────────────────────────────────────────

In [8]:
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [9]:
# transpile the qft quantum circuit for the basis gate of our example backend
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc_dec)
qc_synth.draw(fold=-1)

global phase: 0
                                                                                                                                                                                                                                                                                                                                                                                                                                                               ┌───┐┌─────────────┐                                ┌───┐┌────────────┐                                    ┌───┐┌────────────┐                 ┌───┐┌───────────┐                           ┌───┐    ┌────────────┐                      ┌───┐   ┌───────────┐                                          ┌───┐┌────────────┐             ┌───┐┌───────────┐             ┌───┐┌───────────┐             ┌───┐┌──────────┐            ┌───┐┌───────────┐         ┌───┐   ┌──────────┐┌───┐┌───────────┐   ┌───┐    ┌──────────┐   ┌───┐    ┌──────────┐   ┌───┐    ┌─────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌────┐┌─────────┐
q_0 -> 0 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/1024) ├────────────────────────────────┤ X ├┤ Rz(π/1024) ├────────────────────────────────────┤ X ├┤ Rz(-π/512) ├─────────────────┤ X ├┤ Rz(π/512) ├───────────────────────────┤ X ├────┤ Rz(-π/256) ├──────────────────────┤ X ├───┤ Rz(π/256) ├──────────────────────────────────────────┤ X ├┤ Rz(-π/128) ├─────────────┤ X ├┤ Rz(π/128) ├─────────────┤ X ├┤ Rz(-π/64) ├─────────────┤ X ├┤ Rz(π/64) ├────────────┤ X ├┤ Rz(-π/32) ├─────────┤ X ├───┤ Rz(π/32) ├┤ X ├┤ Rz(-π/16) ├───┤ X ├────┤ Rz(π/16) ├───┤ X ├────┤ Rz(-π/8) ├───┤ X ├────┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├┤ √X ├┤ Rz(π/2) ├
                                                                                                                                                                                                                                                                                                                                ┌───┐┌────────────┐                    ┌───┐   ┌───────────┐                              ┌───┐    ┌────────────┐              └─┬─┘└────┬───┬────┘┌───────────┐         ┌───┐     └─┬─┘├───────────┬┘   ┌───┐   ┌──────────┐             └─┬─┘└────────────┘                 └─┬─┘└───────────┘      ┌───┐┌───────────┐   └─┬─┘    └────────────┘┌───┐ ┌──────────┐    └─┬─┘   └───┬───┬───┘┌───────────┐            ┌───┐┌──────────┐└─┬─┘└───┬───┬────┘ ┌──────────┐└─┬─┘└───┬───┬───┘ ┌─────────┐ └─┬─┘└───────────┘    ┌───┐    └─┬─┘├──────────┤   ┌───┐    └─┬─┘└┬──────────┤┌────┐   └─┬─┘   ├──────────┤└─┬─┘└───────────┘   └─┬─┘    └──────────┘   └─┬─┘    └──────────┘   └─┬─┘    └─────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└────┘└─────────┘
q_2 -> 1 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/256) ├────────────────────┤ X ├───┤ Rz(π/256) ├──────────────────────────────┤ X ├────┤ Rz(-π/128) ├────────────────┼───────┤ X ├─────┤ Rz(π/128) ├─────────┤ X ├───────┼──┤ Rz(-π/64) ├────┤ X ├───┤ Rz(π/64) ├───────────────┼───────────────────────────────────┼─────────────────────┤ X ├┤ Rz(-π/32) ├─────┼────────────────────┤ X ├─┤ Rz(π/32) ├──────┼─────────┤ X ├────┤ Rz(-π/16) ├────────────┤ X ├┤ Rz(π/16) ├──┼──────┤ X ├──────┤ Rz(-π/8) ├──┼──────┤ X ├─────┤ Rz(π/8) ├───┼─────────────

In [10]:
print_qc_characteristics(qc_synth)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [11]:
# Assign a 10-qubit linear CouplingMap to the variable cm
cm = CouplingMap.from_line(num_qubits)

# Route the transpiled quantum circuit
qc_routed = transpile(qc, coupling_map=cm, basis_gates=backend.operation_names)
print("\nRouted circuit:")
print(qc_routed)
print_qc_characteristics(qc_routed)



Routed circuit:
global phase: 1.5723
                                                                              »
q_6 -> 0 ─────────────────────────────────────────────────────────────────────»
                                      ┌───┐┌──────────┐┌───┐┌──────────┐┌────┐»
q_8 -> 1 ─────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├┤ √X ├»
         ┌─────────┐┌────┐┌──────────┐└─┬─┘└──────────┘└─┬─┘├─────────┬┘└────┘»
q_9 -> 2 ┤ Rz(π/2) ├┤ √X ├┤ Rz(3π/4) ├──■────────────────■──┤ Rz(π/8) ├───■───»
         └─────────┘└────┘└──────────┘                      └─────────┘ ┌─┴─┐ »
q_7 -> 3 ───────────────────────────────────────────────────────────────┤ X ├─»
                                                                        └───┘ »
q_4 -> 4 ─────────────────────────────────────────────────────────────────────»
                                                                              »
q_5 -> 5 ─────────────────────────────────────────────────────────────────────»
  

In [12]:
# grade your work!
grade_lab1_ex2(qc_routed)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [13]:
pm_staged = StagedPassManager()
# replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_staged.init = generate_unroll_3q(None)
# initialize the layout stage with an empty pass manager
pm_staged.layout = PassManager()
# set a 'trivial' initial layout, i.e. each qubit in the quantum circuit with index i 
# is mapped to the physical qubit on a device with the same index
pm_staged.layout += TrivialLayout(cm)

# do not modify the next line
pm_staged.layout += generate_embed_passmanager(cm)

In [14]:
from qiskit.transpiler.passes import Optimize1qGates, BarrierBeforeFinalMeasurements

# Specify the basis gates
basis_gates = ['u1', 'u2', 'u3', 'cx']


# Define routing passes
routing_passes = [
    BasicSwap(cm),
    LookaheadSwap(cm),
    StochasticSwap(cm),
]

# Add routing passes to the routing stage
pm_staged.routing = PassManager(routing_passes)

# Add optimization and barrier passes
pm_staged.optimization = PassManager([
    Optimize1qGates(),
    BarrierBeforeFinalMeasurements()
])


# Run the routing with the StagedPassManager
qc_routed = pm_staged.run(qc)

# Print the characteristics of the routed circuit
print("Routed circuit characteristics:")
print_qc_characteristics(qc_routed)

# Draw the routed circuit
print("\nRouted circuit:")
print(qc_routed.draw(fold=-1))


Routed circuit characteristics:
Quantum circuit characteristics
  Depth: 99
  Number of qubits: 10
  Operations: {'swap': 240, 'cp': 45, 'h': 10}
  Number of multi-qubit Operations: 285

Routed circuit:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          ┌───┐
q_0 -> 0 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [15]:
# grade your work!
grade_lab1_ex3(pm_staged)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [ ]:
from qiskit.transpiler import PassManager, StagedPassManager
from qiskit.transpiler.passes import BasisTranslator, CXCancellation,Decompose,StochasticSwap,Unroll3qOrMore, Collect2qBlocks, ConsolidateBlocks, UnitarySynthesis
from qiskit.transpiler.passes import BasicSwap, LookaheadSwap, StochasticSwap

basis_gates = backend.operation_names

init_passes = [
    Unroll3qOrMore()
]

init = PassManager(init_passes)

translate_passes = [
    Collect2qBlocks(),
    ConsolidateBlocks(basis_gates=basis_gates),
    UnitarySynthesis(basis_gates)
]

translate = PassManager(translate_passes)

routing_passes = [
    BasicSwap(cm),
    LookaheadSwap(cm),
    StochasticSwap(cm)
]

routing = PassManager(routing_passes)

pm_staged = StagedPassManager(
    stages=["init", "routing", "translation"],
    init=init,
    routing=routing,
    translation=translate
)

qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)

In [17]:
# grade your work!
grade_lab1_ex4(pm_staged)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [18]:
qk_qc = generate_preset_pass_manager(2, backend=backend).run(qc)
print_qc_characteristics(qk_qc)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [19]:
def noisy_sim(qc, backend):
    # We add measurement operations to the input quantum circuit and then run it on the specified backend
    # A GenericBackendV2 automatically constructs a default model of the expected noise processes, 
    # so backend.run would return noisy simulation results    
    return backend.run(qc.measure_all(inplace=False), shots=7*1024).result().get_counts()

own_transpiler_sim = noisy_sim(qc_routed_synth, backend)
qiskit_transpiler_sim = noisy_sim(qk_qc, backend)
reference_sim =  noisy_sim(transpile(qc.decompose(), backend=backend), BasicSimulator())

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.881
Qiskit transpiler fidelity 0.5568


In [51]:
from qiskit.transpiler.passes import BasisTranslator
from qiskit.circuit import EquivalenceLibrary
pm_opt = StagedPassManager()
basis_gates= backend.operation_names
# Select pass managers and passes for these five stages as you see suitable, note: not every stage must contain a pass.
# Passes are documented at https://docs.quantum.ibm.com/api/qiskit/transpiler_passes

# Replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_opt.init = generate_unroll_3q(None)
# Initialize the layout stage with an empty pass manager
pm_opt.layout = PassManager()
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/layout for potential layouting passes
pm_opt.layout += TrivialLayout(cm)
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing for potential routing passes
pm_opt.routing = PassManager(StarPreRouting())
pm_opt.routing += SabreSwap(cm,heuristic="decay")

# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis for potential translation passes
pm_opt.translation = PassManager()
pm_opt.translation += BasisTranslator(SessionEquivalenceLibrary,basis_gates)
pm_opt.translation +=HighLevelSynthesis()
#pm_opt.translation +=UnitarySynthesis()

pm_opt.optimization = PassManager()
pm_opt.optimization += Collect2qBlocks()
pm_opt.optimization += ConsolidateBlocks()
pm_opt.optimization += UnitarySynthesis(basis_gates)
    
     
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/optimization for potential optimization passes
# scheduling is not needed in this lab - we will not run the quantum circuit on a real device
# pm_opt.scheduling = None

# do not modify the next line
pm_opt.layout += generate_embed_passmanager(cm)

qc_opt = pm_opt.run(qc)

In [52]:
print_qc_characteristics(qc_opt)
reduction_ratio = round(100-100*(get_qc_characteristics(qc_opt)['num_multi_qubit_ops']/get_qc_characteristics(qk_qc)['num_multi_qubit_ops']), 3)
print("Reduction in two-qubit gates compared to qiskit {}%!".format(reduction_ratio))
qc_opt.draw(fold=-1)

Quantum circuit characteristics
  Depth: 182
  Number of qubits: 10
  Operations: {'rz': 398, 'sx': 185, 'cx': 134, 'x': 75}
  Number of multi-qubit Operations: 134
Reduction in two-qubit gates compared to qiskit -48.889%!


global phase: 3.9285
           ┌──────────┐  ┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ┌───┐┌──────────┐          ┌───┐┌────────────┐┌───┐  ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
q_0 -> 0 ──┤ Rz(-π/4) ├──┤ √X ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/2) ├──────────┤ X ├┤ Rz(1.5677) ├┤ X ├──┤ Rz(π/4) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          ┌┴──────────┤  ├────┤┌────────┐                                                                                                                                                                                                              

In [53]:
# grade your work!
grade_lab1_ex5(pm_opt)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [ ]:
from qiskit.transpiler import AnalysisPass
from qiskit.circuit import QuantumCircuit
from qiskit.dagcircuit import DAGCircuit
from qiskit.circuit.quantumregister import Qubit
from typing import Dict

class GatesPerQubit(AnalysisPass):
    def __init__(self):
        super().__init__()

    def run(self, dag: DAGCircuit):
        """
        Runs the analysis pass on the given DAG circuit.
        
        Args:
            dag (DAGCircuit): The directed acyclic graph representation of the quantum circuit.
        """
        one_q_op = {}
        two_q_op = {}

        # Initialize the dictionaries with all qubits set to 0
        for qubit in dag.qubits:
            one_q_op[qubit] = 0
            two_q_op[qubit] = 0

        # Iterate through all operations in the DAG
        for node in dag.op_nodes():
            qargs = node.qargs
            
            if len(qargs) == 1:
                # Single-qubit gate
                qubit = qargs[0]
                one_q_op[qubit] += 1
            elif len(qargs) == 2:
                # Two-qubit gate
                for qubit in qargs:
                    two_q_op[qubit] += 1

        # Store the results in the property set
        self.property_set['one_q_op'] = one_q_op
        self.property_set['two_q_op'] = two_q_op

    def supported(self, property_set: Dict):
        """
        Check if the pass is supported.
        
        Args:
            property_set (Dict): The property set.
        
        Returns:
            bool: Always True for this pass as it doesn't have any specific requirements.
        """
        return True

In [ ]:
# grade your work!
grade_lab1_ex6(GatesPerQubit)

In [ ]:
qc = QFT(4, do_swaps=False)
qc = generate_preset_pass_manager(2, backend=backend).run(qc)
gpq = GatesPerQubit()
gpq(qc)
print("single-qubit gates on qubit", gpq.property_set["one_q_op"])
print("two-qubit gates on qubit", gpq.property_set["two_q_op"])
qc.draw(fold=-1, idle_wires=False)

In [ ]:
pg = Gate('Peres', 3, params=[], label='PG')

In [ ]:
qc_pg = QuantumCircuit(3)
qc_pg.append(pg, [0, 1, 2])
qc_pg.draw()

In [ ]:
def get_qc_in(nq):
    # QFT circuit, feel free to use a previously defined pass manager for the QFT circuit
    qc_qft = QFT(nq, do_swaps=False)
    # part of the circuit including the Peres gate
    qc_inner = QuantumCircuit(nq)
    for i in range(1, nq - 1):
        qc_inner.append(pg, [nq - i - 2, nq - i - 1, nq - 1])

    qc_in = QuantumCircuit(nq)
    # add QFT circuit to qc_in
    qc_in.compose(qc_qft, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)

    qc_in.rz(pi, nq - 1)
    # add circuit with peres gates
    qc_in.compose(qc_inner, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)
    # add inverse QFT circuit
    qc_in.compose(qc_qft.inverse(), range(nq), inplace=True)
    return qc_in

nq = 5
qc_in = get_qc_in(nq)
qc_in.draw(fold=-1)

In [ ]:
from qiskit.transpiler import TransformationPass
from qiskit.circuit import QuantumCircuit
from qiskit.dagcircuit import DAGCircuit
from qiskit.converters import circuit_to_dag
from math import pi

class PeresGateTranslation(TransformationPass):
    def __init__(self):
        super().__init__()

    def get_peres_decomposition(self):
        qcsx = QuantumCircuit(2)
        qcsx.rz(pi / 4, 0)
        qcsx.rz(pi / 2, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)
        qcsx.cx(0, 1)
        qcsx.rz(-pi / 4, 1)
        qcsx.cx(0, 1)
        qcsx.rz(3 * pi / 4, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)

        qcsx_inv = QuantumCircuit(2)
        qcsx_inv.rz(pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 0)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 4, 0)

        qc_dec = QuantumCircuit(3)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx_inv, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(0, 1)
        return qc_dec

    def run(self, dag: DAGCircuit) -> DAGCircuit:
        """
        Runs this pass on the provided DAG circuit, replacing Peres gates with their decomposition.

        Args:
            dag (DAGCircuit): The DAG circuit to transform.

        Returns:
            DAGCircuit: The transformed DAG circuit.
        """
        peres_decomposition = self.get_peres_decomposition()
        peres_dag = circuit_to_dag(peres_decomposition)

        for node in dag.op_nodes():
            if node.name == 'Peres':
                qubits = node.qargs
                if len(qubits) != 3:
                    raise ValueError(f"Peres gate must act on 3 qubits, but found {len(qubits)}")

                # Create a mapping from the decomposition qubits to the circuit qubits
                qubit_map = {peres_dag.qubits[i]: qubits[i] for i in range(3)}

                # Replace the Peres gate with its decomposition
                dag.substitute_node_with_dag(node, peres_dag, qubit_map)

        return dag


In [ ]:
# grade your work!
grade_lab1_ex7(PeresGateTranslation, pg)